In [ ]:
from openai import OpenAI

client = OpenAI() # defautls to getting the key using os.environ.get("OPENAI_API_KEY")

In [ ]:
# Upload LinkedIn profile to OpenAI
profile = client.files.create(
  file=open("LinkedIn_profile.json", "rb"),
  purpose="assistants"
)
print(profile)

In [ ]:
# read in contents of job description
with open("job_description.json", "r") as f:
    job_description = f.read()
print(job_description)

In [ ]:
# Attach the example linkedIn profile to this thread
# OpenAI automatically creates a vector store for the file that expires in 7 days
thread = client.beta.threads.create(
    tool_resources={
        'file_search': {
            'vector_stores': [
                {
                    'file_ids': [profile.id]
                }
            ]
        }
    },
    messages=[
        {
            'role': 'user',
            'content': job_description
        }
    ]
)

print(thread)

In [ ]:
import os

def run_assistant(thread_id):
    return client.beta.threads.runs.create_and_poll(
        thread_id=thread_id,
        assistant_id=os.getenv("OPENAI_ASSISTANT_ID")
    )

print(run_assistant(thread.id))

In [ ]:
import json

def extract_resume(thread_id):
    thread_messages = client.beta.threads.messages.list(thread_id)
    asssitant_response = thread_messages.data[0].content[0].text.value

    # Extract the JSON resume from the message content
    start = asssitant_response.index("```json") + len("```json")
    end = asssitant_response.index("```", start)

    resume = json.loads(asssitant_response[start:end])

    # Remove endDate if empty (indicates current position)
    for experience in resume["work"]:
        if experience.get("endDate") == "":
            experience.pop("endDate")
    
    return resume

resume = extract_resume(thread.id)
print(resume)

In [ ]:
from jsonschema import validate
import json

# read in schema
with open("schema.json", "r") as f:
    schema = json.loads(f.read())
print(schema)

# Validate Schema
def validate_and_correct_resume(resume):
    try:
        validate(instance=resume, schema=schema)
    except Exception as e:
        print(f'Error validating JSON Schema: {e}')
        
        # Feed error back into assistant
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=f'There was an error validating the generated JSON resume against the JSON Schema.\
                Fix the issues outlined by the following error and regenerate the JSON resume: {e}'
        )

        run_assistant(thread.id)
        resume = validate_and_correct_resume(extract_resume(thread.id))
    
    return resume

resume = validate_and_correct_resume(resume)

In [ ]:
# write custom resume to file
with open('custom_resume.json', 'w') as output_file:
    json.dump(resume, output_file)